In [23]:
%reset -f

import sys
sys.path.append("../data/exadata/parquet_dataset/query_tool")

import os
import pandas as pd
import numpy as np
import itertools
import datetime
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.linear_model import LinearRegression
from query_tool import M100DataClient
from matplotlib import pyplot as plt
from models import *

dataset_path = "../data/m100"
processed_path = "../data/processed_m100"
client = M100DataClient(dataset_path)



In [7]:
#TODO FOR BOTH: TIMESTAMP INDICES

def extract_node_values(feature_list, n_rows, nodes_of_interest, columns, year_month):
    node_dfs = []
    for f in feature_list: 
        query_df = client.query(f, 
                      columns=columns,
                      year_month=year_month)   
        query_df.dropna(inplace=True)
        node_list = query_df["node"].to_numpy().astype(np.int32)
        for noi in nodes_of_interest:
            idxs = np.where(node_list == noi, True, False)
            nodes = query_df.iloc[idxs].iloc[:n_rows]
            node_values = nodes["value"].values
            
            node_df = pd.DataFrame(index=nodes["timestamp"])
            new_name = str(noi) + "_" + str(f)
            node_df[new_name] = nodes["value"].values
            node_dfs.append(node_df)
            
    plugin_df=pd.concat(node_dfs, axis=1, join="inner")
    return plugin_df

def extract_values(feature_list, n_rows, columns, year_month):
    dfs = []
    for f in feature_list: 
        query_df = client.query(f, 
                      columns=columns,
                      year_month=year_month)   
        query_df.dropna(inplace=True)
        df = pd.DataFrame(index=query_df["timestamp"])
        df[f] = query_df["value"].values
        df = df.iloc[:n_rows]
        dfs.append(df)
        
    plugin_df=pd.concat(dfs, axis=1, join="inner")
    return plugin_df
    
#ipmi collects physical/hardware properties of nodes
def make_ipmi():
    ipmi_features = ["ambient"]
    ipmi_columns = ["timestamp", "node", "value"]
    ipmi_df = extract_node_values(ipmi_features, n_rows, nodes_of_interest, ipmi_columns, year_month)
    return ipmi_df

#ganglia collects metrics on utilization of hardware 
def make_ganglia():
    ganglia_features = []
    ganglia_columns = ["timestamp", "node", "value"]
    ganglia_df = extract_node_values(ganglia_features, n_rows, nodes_of_interest,ganglia_columns, year_month)
    return ganglia_df
    
#vertiv plugin collects data for AC cooling 

def make_vertiv():
    vertiv_features = ["Supply_Air_Temperature"]
    vertiv_columns = ["timestamp", "value"]
    vertiv_df = extract_values(vertiv_features, n_rows, vertiv_columns, year_month)
    return vertiv_df

def make_vertiv_labels():
    vertiv_labels = ["Return_Air_Temperature"]
    vertiv_columns = ["timestamp", "value"]
    vertiv_labels = extract_values(vertiv_labels, n_rows, vertiv_columns, year_month)
    return vertiv_labels
   

#weather data
def make_weather():
    weather_features = ["temp"]
    weather_columns = ["timestamp", "value"]
    weather_df = extract_values(weather_features, n_rows, weather_columns, year_month)
    return weather_df

In [10]:
"""Cell purpose: pull desired features from plugins"""


n_rows = 1000000
nodes_of_interest = [i for i in range(10)]
# nodes_of_interest = [0]
year_month = ["22-02"]

vertiv_df = make_vertiv()
vertiv_df = vertiv_df.groupby(vertiv_df.index).mean()
vertiv_df.to_csv(os.path.join(processed_path, "vertiv.csv"))


vertiv_labels = make_vertiv_labels()
vertiv_labels.to_csv(os.path.join(processed_path, "vertiv_labels.csv"))

ipmi_df = make_ipmi()
ipmi_df.to_csv(os.path.join(processed_path, "ipmi.csv"))

weather_df = make_weather()
weather_df.to_csv(os.path.join(processed_path, "weather.csv"))


#schneider collects data on liquid cooling system
# schneider_df = client.query(["Tot"],
#                   columns=['timestamp', 'value'],
#                   year_month=["22-02"])

# logics collects power consumption data from equipment and devices
# logics_features = ["Tot"]
# logics_columns = ["timestamp", "node", "value"]
# logics_df = extract_node_values(logics_features, n_rows, nodes_of_interest, logics_columns, year_month)
# logics_df.to_csv(os.path.join(processed_path, "logics.csv"))
# del logics_df #probably done anyway by python but no harm

Retrieving data of type: float
Retrieving data of type: float
Retrieving data of type: float
Retrieving data of type: float


In [11]:
weather_df = pd.read_csv(os.path.join(processed_path, "weather.csv"), index_col = "timestamp")
vertiv_df = pd.read_csv(os.path.join(processed_path, "vertiv.csv"), index_col="timestamp")
vertiv_labels = pd.read_csv(os.path.join(processed_path, "vertiv_labels.csv"),index_col="timestamp")
# ganglia_df= pd.read_csv(os.path.join(processed_path, "ganglia.csv"))
ipmi_df = pd.read_csv(os.path.join(processed_path, "ipmi.csv"), index_col="timestamp")


plugin_dfs = [ipmi_df, weather_df, vertiv_df] #start w/ small one for now
df = plugin_dfs[0]
for d in plugin_dfs[1:]:
    df = df.merge(d, left_index=True, right_index=True)
df
#current issue: it seems like there are duplicate timestamps in supply air temp. Solution would be to average values when there are duplicate timestamps

,0_ambient,1_ambient,2_ambient,3_ambient,4_ambient,5_ambient,6_ambient,7_ambient,8_ambient,9_ambient,temp,Supply_Air_Temperature
timestamp,,,,,,,,,,,,
2022-02-06 23:00:00+00:00,21.8,20.8,21.6,21.6,22.4,22.6,22.6,23.0,23.6,24.0,7.23,19.950000
2022-02-07 01:20:00+00:00,22.2,21.4,22.0,22.2,22.8,22.6,22.4,22.8,23.4,23.8,4.47,19.233334
2022-02-07 03:20:00+00:00,21.2,20.0,21.2,21.6,22.4,22.2,22.2,22.6,23.0,23.4,4.27,19.383333
2022-02-07 03:30:00+00:00,21.2,20.2,21.4,21.6,22.4,22.2,22.2,22.6,23.0,23.4,4.41,19.033333
2022-02-07 06:00:00+00:00,21.4,20.6,21.4,21.4,22.0,22.0,21.8,22.2,22.8,23.2,4.31,19.066668
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-11 22:10:00+00:00,20.6,19.4,20.8,21.0,22.0,21.8,21.8,22.4,22.8,23.2,8.02,20.300000
2022-02-11 22:20:00+00:00,21.2,20.4,21.2,21.4,22.2,22.2,22.0,22.6,23.0,23.4,7.96,19.850000
2022-02-11 22:30:00+00:00,21.2,20.4,21.2,21.2,21.8,22.0,22.0,22.4,22.8,23.2,7.80,20.000000


In [13]:
df.index[0]

'2022-02-06 23:00:00+00:00'

In [20]:
"""Cell purpose: preprocess and/or merge data as desired"""
# 
# feature_list = itertools.chain(ipmi_features, weather_features, vertiv_features)
# label_list = itertools.chain(vertiv_labels) 


feature_list = ["0_ambient"]
label_list = ["1_ambient"] #not what we actually want, but for testing
feature_df = df[feature_list] 
label_df = df[label_list]

#align t and t+1
feature_df = feature_df.iloc[:-1]
label_df = label_df.iloc[1:]

assert feature_df.shape[0] == label_df.shape[0]



In [21]:
# Will have to make work with timestamp indexign later
class M100Data(Dataset):
    def __init__(self, feature_df, label_df, transform = None, label_transform = None):
        self.feature_df = feature_df
        self.label_df = label_df
        self.transform = transform
        self.feature_dim = self.feature_df.shape[1]
        self.label_dim = self.label_df.shape[1]
    def __len__(self):
        return self.feature_df.shape[0]

    def __getitem__(self, idx):
        feature = torch.as_tensor(self.feature_df.iloc[idx].values)
        label = torch.as_tensor(self.label_df.iloc[idx].values)
        return feature, label
    
train_ds = M100Data(feature_df, label_df)
trainloader = DataLoader(train_ds, batch_size = 8, shuffle = True)

In [22]:
epochs = 1
model = LinearNN(train_ds.feature_dim, train_ds.label_dim)
optimizer = torch.optim.SGD(model.parameters())
loss_fn = torch.nn.CrossEntropyLoss()
for e in epochs:
    train_loss = train_epoch(model, trainloader, optimizer, loss_fn, epochs)
    # eval_loss = eval_epoch(model, evalloader, loss_fn)

AttributeError: cannot assign module before Module.__init__() call

In [ ]:
"""Cell purpose: train"""
# model = LinearRegression().fit(merged_df["air_temp"].to_numpy().reshape(-1,1), merged_df["ambient_temp"].to_numpy().reshape(-1,1))

In [ ]:
"""Cell purpose: inference"""
# preds = model.predict(test_df["air_temp"].to_numpy().reshape(-1,1))


In [ ]:
"""Cell purpose: benchmarking"""

# labels = test_df["ambient_temp"].to_numpy().reshape(-1,1)
# 
# 
# timesteps = [t for t in range(preds.shape[0])]
# colors = iter(plt.cm.rainbow(np.linspace(0, 0.5, 2)))
# plt.scatter(preds, labels, c=["blue", "red"])
# 
# plt.legend(loc="upper left")
# plt.xlabel("Time")
# plt.ylabel("Ambient temperature")
# plt.grid()
# plt.show()



In [ ]:
""
